In [6]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [7]:
import pandas as pd
from urlparse import urlparse
import tldextract
from collections import Counter
from time import time

In [8]:
huizhou_stop_words = [u'的', u'了', u'在', u'是', u'安徽', u'和', u'有', u'我', u'年', u'上', u'他', u'我们', 
                      u'为', u'与', u'人', u'也', u'于', u'中', u'就', u'都', u'在线', u'等', u'不', u'网站', u'你', 
                      u'一个', u'到', u'之', u'对', u'说', u'进入', u'后', u'月', u'将', u'时', u'又', u'被', u'会员'
                     u'版权', u'论坛', u'以', u'而', u'着', u'下', u'这', u'但', u'要', u'或', u'首页', u'这',
                     u'地', u'从', u'卡', u'来', '多', u'日', u'她', u'还', u'联系', u'那', u'一处', u'一直', u'个', 
                     u'向', u'并', u'曾', u'这样', u'里', u'合肥', u'推荐', u'专栏', u'相关', u'当前', u'所有', u'中心'
                     u'频道', u'编辑', u'位置', u'第一', u'社区', u'看', '中安', u'安徽省', u'大', u'许可证', u'许可',
                     u'让', u'没有', u'什么', u'其', '请', '该', '用', '积分', u'凤阳', u'一篇', u'版权', u'县',
                     ]

In [9]:
df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/segments/huizhou_Solr_c_v1.csv')

In [10]:
df.info()

In [11]:
new_df = df[['url', 'domain',  'title', 'content', 'content_seg']]

In [12]:
new_df.head()

,url,domain,title,content,content_seg
0,http://cul.anhuinews.com/system/2016/11/28/007...,anhuinews.com,洪建华夫妇的竹刻人生,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质文化遗产·徽...,谁 能 想到 ， 洪 建华 夫妇 这 对 一 无 学历 、 二 无 家传 、 三无 师承 的...
1,http://cul.anhuinews.com/system/2016/11/21/007...,anhuinews.com,徽州民歌传人操明花校园授艺,近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州区岩寺镇中心...,近日 ， 省非 物质 文化遗产 徽州 民歌 传承 人操 明花 前往 休宁县 海阳 二小 、 ...
2,http://cul.anhuinews.com/system/2016/11/03/007...,anhuinews.com,黄山楹联漫谈,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵不上黄山峰林...,黄山 楹联 第一 人当 推 明朝 诗人 余绍祉 。 他 遍历 祖国 名山大川 ， 见到 无数...
3,http://cul.anhuinews.com/system/2016/01/26/007...,anhuinews.com,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,徽州 民居 、 家具 木板 和 圆木 雕刻 艺术 的 简称 。 木雕 在 旧属 徽州 各县 ...
4,http://cul.anhuinews.com/system/2016/01/26/007...,anhuinews.com,徽派雕刻——徽州砖雕艺术,徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入密烧制，嵌窃时再进行修...,徽州 砖雕 徽州 砖雕 的 造型 ， 来源于 汉 画象 砖 。 汉 画象 砖 大都 是 模印...


In [15]:
df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/segments/huizhou_non_Solr_c_v1.csv')

In [16]:
df.info()

In [17]:
joined_df = df[['url', 'domain', 'title', 'content', 'content_seg']]

In [18]:
ret_df = joined_df.append(new_df)

In [19]:
ret_df.info()

In [20]:
ret_df = ret_df.drop_duplicates(['url'])

In [21]:
ret_df.info()

In [22]:
#drop any duplicate content directly from 198213 to 196890
df = ret_df.drop_duplicates(['content'])

In [23]:
df.info()

In [24]:
#content ready for next step (tot. 169086)
df.to_csv('/Users/nhu2000/projects/SimilarSearch/data/final/huizhou-final-v1.csv', encoding='utf-8', index=False)

In [25]:
import unicodedata
def remove_non_chinese(s):
    r = [c for c in s if unicodedata.category(c).startswith('L')]
#print r
    return ''.join(r)
#print remove_non_chinese(s)

In [26]:
def ch_title_letters_removal(x):
    #print 'coming', x
    t = x.strip('  ')
    if(len(t)) == 0:
        return None
    
    if(len(x) > 0):
        #s = x.strip()
        s = remove_non_chinese(x)
        if len(s) > 0 and s not in huizhou_stop_words:
            return s.strip()
        else:
            return None
    return None

In [27]:
global idx
idx = 0
def map_signature_vect(x): 
    global idx
    print idx
    idx += 1
    s = x.decode('utf-8').split()
    #print 'vect ---', len(s), s
    ret = [ch_title_letters_removal(item) for item in s if len(item) > 0 and ch_title_letters_removal(item) != None]
    #print 'ret ---', ret, len(ret)
    count = Counter(ret)
    
    #print 'count', len(count)
    
    top_50 = count.most_common(50)
        
    #print 'top_100', len(top_100)
    
    if len(top_50) > 0:
        #print 'going ---'
        words, cnt = zip(*top_50)
        tot = float(sum(cnt))
        fq = [item/tot for item in cnt]
        return dict(zip(words, fq))
    else:
        #print 'warning'
        return None

In [103]:
def get_summary(x):
    sentences = x.split('\n')
    #print 'len -----', len(sentences)
    #skip the first sentences
    if len(sentences) > 3:
        s = '。'.join(sentences[1:]).decode('utf-8')
    else:
        s = x.strip().decode('utf-8')
    if len(s) > 50:
        ret = ''.join(s[:50]) + '...'
    else:
        ret = s
    
    if len(s) < 10:
        ret = x
        
    return ret.strip()

map_signature_vect(df.iloc[1]['content_seg'])

In [36]:
df.info()

In [30]:
df['s_vec'] = df.content_seg.map(lambda x: map_signature_vect(x))

In [33]:
df['s_vec_str'] = df.s_vec.map(lambda x: str(x))

In [34]:
df_no_dup = df.drop_duplicates(['s_vec_str']) #drop from 169086 to 167442

In [35]:
df_no_dup.info()

In [104]:
df['summary'] = df.content.map(lambda x: get_summary(x))

In [112]:
#check summary
for i in range(8000, 9000): #df(df)):
    s = df.iloc[i]['summary']
    r = df.iloc[i]['content']
    url = df.iloc[i]['url']
    print '------------------'
    print i, s
    
    #print url

In [119]:
for i in range(0, 1000): #df(df)):
    r = df.iloc[i]['content']
    url = df.iloc[i]['url']
    print '------------------'
    print i, r

In [115]:
df_slim = df[['url', 'title', 'summary', 's_vec']]

In [116]:
df_slim.info()

In [117]:
df_slim.to_csv('/Users/nhu2000/projects/SimilarSearch/data/merged/huizhou-slim_v0_1.csv', encoding='utf-8', index=False)

In [84]:
df_slim.head()

,url,title,summary,s_vec
0,http://cul.anhuinews.com/system/2016/11/28/007...,洪建华夫妇的竹刻人生,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质文化遗产·徽...,"{u'补助': 0.00731707317073, u'竹木': 0.00975609756..."
1,http://cul.anhuinews.com/system/2016/11/21/007...,徽州民歌传人操明花校园授艺,近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州区岩寺镇中心...,"{u'原生': 0.0113636363636, u'它': 0.0113636363636..."
2,http://cul.anhuinews.com/system/2016/11/03/007...,黄山楹联漫谈,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵不上黄山峰林...,"{u'气派': 0.0119047619048, u'名句': 0.011904761904..."
3,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,"{u'吉祥图案': 0.0169491525424, u'少数': 0.0169491525..."
4,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州砖雕艺术,徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入密烧制，嵌窃时再进行修...,"{u'一': 0.00961538461538, u'它': 0.0192307692308..."


In [122]:
new_df = df[['url', 'title', 'summary', 'content', 'content_seg', 's_vec']]

In [123]:
new_df.to_csv('/Users/nhu2000/projects/SimilarSearch/data/merged/huizhou-v0_1.csv', encoding='utf-8', index=False)